# Residual Gated Graph ConvNets
### Xavier Bresson, Jan. 15 2018

In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import pdb 
import time
import numpy as np
import pickle

if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    #torch.cuda.manual_seed(1)
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    #torch.manual_seed(1)
    
import core.block as block
import core.graph_generator as g
from core.GraphConvNet import GraphConvNet

from sklearn.metrics import confusion_matrix

cuda not available
cuda not available


In [2]:

#################
# select task and task parameters
#################


# subgraph matching
if 2==1:
    task_parameters = {}
    task_parameters['flag_task'] = 'matching'
    task_parameters['nb_communities'] = 10
    task_parameters['nb_clusters_target'] = 2
    task_parameters['Voc'] = 3
    task_parameters['size_min'] = 15
    task_parameters['size_max'] = 25
    task_parameters['size_subgraph'] = 20
    task_parameters['p'] = 0.5
    task_parameters['q'] = 0.1
    task_parameters['W0'] = block.random_graph(task_parameters['size_subgraph'],task_parameters['p'])
    task_parameters['u0'] = np.random.randint(task_parameters['Voc'],size=task_parameters['size_subgraph'])

    
# semi-supervised clustering
if 1==1:
    task_parameters = {}
    task_parameters['flag_task'] = 'clustering'
    task_parameters['nb_communities'] = 10
    task_parameters['nb_clusters_target'] = task_parameters['nb_communities']
    task_parameters['Voc'] = task_parameters['nb_communities'] + 1
    task_parameters['size_min'] = 5
    task_parameters['size_max'] = 25
    task_parameters['p'] = 0.5
    task_parameters['q'] = 0.1  
    
#print(task_parameters)

In [3]:

#################
# network and optimization parameters
#################

# network parameters
net_parameters = {}
net_parameters['Voc'] = task_parameters['Voc']
net_parameters['D'] = 50
net_parameters['nb_clusters_target'] = task_parameters['nb_clusters_target']
net_parameters['H'] = 50
net_parameters['L'] = 10
#print(net_parameters)


# optimization parameters
opt_parameters = {}
opt_parameters['learning_rate'] = 0.00075   # ADAM
opt_parameters['max_iters'] = 5000   
opt_parameters['batch_iters'] = 100
if 2==1: # fast debugging
    opt_parameters['max_iters'] = 101 
    opt_parameters['batch_iters'] = 10
opt_parameters['decay_rate'] = 1.25   
#print(opt_parameters)


In [4]:
#print(result)

## Load model and test

In [5]:
net_parameters

{'Voc': 11, 'D': 50, 'nb_clusters_target': 10, 'H': 50, 'L': 10}

In [6]:
net = GraphConvNet(net_parameters, task_parameters)


nb of hidden layers= 10
dim of layers (w/ embed dim)= [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]




In [7]:
filename = 'data/checkpoint.pth.tar'
checkpoint = torch.load(filename)
net.load_state_dict(checkpoint['state_dict'])
#net.eval()

In [8]:
task_parameters['flag_task'] = 'clustering'
task_parameters['nb_communities'] = 10
task_parameters['nb_clusters_target'] = task_parameters['nb_communities']
task_parameters['Voc'] = task_parameters['nb_communities'] + 1
task_parameters['size_min'] = 20
task_parameters['size_max'] = 30

In [9]:
test_x = g.graph_semi_super_clu(task_parameters)
test_y = test_x.target
test_y = Variable( torch.LongTensor(test_y).type(dtypeLong) , requires_grad=False) 

print(test_x.adj_matrix.shape)

(248, 248)


In [10]:
# forward, loss
y = net.forward(test_x)
labels = test_y.data.cpu().numpy()

torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor
torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor
torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor
torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor
torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor


/Users/signapoop/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [11]:
C = np.argmax( torch.nn.Softmax(dim=0)(y).data.cpu().numpy() , axis=1)

In [12]:
C

array([3, 3, 3, 2, 3, 1, 3, 5, 8, 0, 0, 3, 1, 4, 8, 2, 0, 7, 6, 6, 7, 4,
       8, 1, 0, 2, 3, 5, 4, 1, 4, 4, 8, 8, 9, 7, 5, 6, 6, 1, 5, 5, 3, 8,
       9, 8, 8, 2, 6, 2, 6, 6, 9, 7, 1, 8, 4, 2, 0, 7, 8, 9, 9, 3, 2, 9,
       0, 4, 4, 6, 7, 9, 6, 0, 5, 4, 2, 5, 8, 1, 6, 0, 3, 9, 1, 1, 2, 5,
       0, 4, 8, 9, 0, 6, 4, 7, 8, 1, 1, 3, 2, 0, 5, 6, 9, 4, 4, 7, 1, 5,
       6, 8, 1, 6, 7, 8, 5, 9, 4, 2, 8, 2, 4, 0, 1, 7, 9, 5, 7, 8, 2, 3,
       2, 9, 2, 6, 9, 7, 5, 4, 7, 5, 1, 5, 0, 3, 4, 8, 0, 6, 0, 0, 0, 8,
       3, 8, 2, 7, 6, 5, 5, 7, 6, 7, 2, 5, 9, 6, 8, 1, 5, 5, 9, 4, 3, 1,
       5, 0, 3, 3, 5, 9, 5, 1, 2, 6, 1, 6, 8, 7, 1, 4, 9, 0, 8, 2, 2, 7,
       3, 5, 5, 6, 3, 8, 5, 6, 9, 7, 5, 7, 8, 8, 1, 0, 6, 1, 4, 9, 6, 5,
       4, 0, 4, 1, 0, 4, 4, 3, 4, 3, 9, 7, 4, 9, 3, 1, 4, 0, 8, 0, 1, 2,
       9, 3, 4, 4, 1, 1])

In [13]:
S = test_y.data.cpu().numpy()

In [14]:
S

array([3, 3, 3, 2, 3, 1, 3, 5, 8, 0, 0, 3, 1, 4, 8, 2, 0, 7, 6, 6, 7, 4,
       8, 1, 0, 2, 3, 5, 4, 1, 4, 4, 8, 8, 7, 7, 5, 6, 6, 1, 5, 5, 3, 8,
       9, 8, 8, 2, 6, 2, 6, 6, 9, 7, 1, 8, 4, 2, 0, 7, 8, 9, 9, 3, 2, 9,
       0, 4, 4, 6, 7, 9, 6, 0, 5, 4, 2, 5, 8, 1, 6, 0, 3, 9, 1, 1, 2, 5,
       0, 4, 8, 7, 0, 6, 4, 7, 8, 1, 1, 3, 2, 0, 5, 6, 9, 4, 4, 7, 1, 5,
       6, 8, 1, 6, 7, 8, 5, 9, 4, 2, 8, 2, 4, 0, 1, 7, 9, 5, 7, 8, 2, 3,
       2, 9, 2, 6, 9, 7, 5, 4, 7, 5, 1, 8, 0, 3, 4, 8, 0, 6, 0, 0, 0, 8,
       3, 8, 2, 7, 6, 5, 5, 7, 6, 7, 2, 5, 9, 6, 8, 1, 5, 5, 9, 4, 3, 1,
       5, 0, 3, 3, 5, 9, 5, 1, 2, 6, 1, 6, 8, 7, 1, 4, 9, 0, 8, 2, 2, 7,
       3, 5, 5, 6, 3, 8, 5, 6, 9, 7, 5, 7, 8, 8, 1, 0, 6, 1, 4, 9, 6, 5,
       4, 0, 4, 1, 0, 4, 4, 3, 4, 3, 9, 7, 4, 9, 3, 1, 4, 0, 8, 0, 1, 2,
       9, 3, 4, 4, 1, 1])

In [15]:
# CM = confusion_matrix(y_true=S, y_pred=C).astype(np.float32)
# nb_classes = CM.shape[0]
# for r in range(nb_classes):
#     cluster = np.where(S==r)[0]
#     CM[r,:] /= cluster.shape[0]

In [16]:
CM = confusion_matrix(y_true=S, y_pred=C)
CM

array([[24,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 27,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 21,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 23,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 29,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 27,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 25,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 21,  0,  2],
       [ 0,  0,  0,  0,  0,  1,  0,  0, 27,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 21]])

In [17]:
E_start = test_x.edge_to_starting_vertex
E_start = torch.from_numpy(E_start.toarray()).type(dtypeFloat)

In [18]:
E_start.shape

torch.Size([8810, 248])

In [19]:
E_end = test_x.edge_to_ending_vertex 
E_end = torch.from_numpy(E_end.toarray()).type(dtypeFloat)
E_end.shape

torch.Size([8810, 248])

In [20]:
test_x.target.numpy()

array([3, 3, 3, 2, 3, 1, 3, 5, 8, 0, 0, 3, 1, 4, 8, 2, 0, 7, 6, 6, 7, 4,
       8, 1, 0, 2, 3, 5, 4, 1, 4, 4, 8, 8, 7, 7, 5, 6, 6, 1, 5, 5, 3, 8,
       9, 8, 8, 2, 6, 2, 6, 6, 9, 7, 1, 8, 4, 2, 0, 7, 8, 9, 9, 3, 2, 9,
       0, 4, 4, 6, 7, 9, 6, 0, 5, 4, 2, 5, 8, 1, 6, 0, 3, 9, 1, 1, 2, 5,
       0, 4, 8, 7, 0, 6, 4, 7, 8, 1, 1, 3, 2, 0, 5, 6, 9, 4, 4, 7, 1, 5,
       6, 8, 1, 6, 7, 8, 5, 9, 4, 2, 8, 2, 4, 0, 1, 7, 9, 5, 7, 8, 2, 3,
       2, 9, 2, 6, 9, 7, 5, 4, 7, 5, 1, 8, 0, 3, 4, 8, 0, 6, 0, 0, 0, 8,
       3, 8, 2, 7, 6, 5, 5, 7, 6, 7, 2, 5, 9, 6, 8, 1, 5, 5, 9, 4, 3, 1,
       5, 0, 3, 3, 5, 9, 5, 1, 2, 6, 1, 6, 8, 7, 1, 4, 9, 0, 8, 2, 2, 7,
       3, 5, 5, 6, 3, 8, 5, 6, 9, 7, 5, 7, 8, 8, 1, 0, 6, 1, 4, 9, 6, 5,
       4, 0, 4, 1, 0, 4, 4, 3, 4, 3, 9, 7, 4, 9, 3, 1, 4, 0, 8, 0, 1, 2,
       9, 3, 4, 4, 1, 1])

In [21]:
# Extract embeddings

In [22]:
all_X = [net.tracker[i].detach().numpy() for i in range(len(net.tracker))]
y = test_x.target.numpy()
import pickle
filename = 'data/embedding.pkl'
with open(filename, 'wb') as f:
    pickle.dump([all_X, y], f)

In [23]:
print(all_X[0].shape)
print(y.shape)

(248, 50)
(248,)


In [24]:
y = net.forward(test_x)
y.shape

torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor
torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor
torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor
torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor


/Users/signapoop/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.FloatTensor
torch.Size([248, 50])
torch.FloatTensor


torch.Size([248, 10])

In [25]:
test_x.signal

tensor([ 4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  3,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  7,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 